In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-category-dataset/News_Category_Dataset_v3.json


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel, linear_kernel

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
df = pd.read_json("/kaggle/input/news-category-dataset/News_Category_Dataset_v3.json",
                        lines=True,
                        orient='columns')


In [5]:
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [6]:
df = df[:40000]

In [7]:
# df_train = df[:(df.shape[0])//2]
# df_val = df[((df.shape[0])//2)+1:]

In [8]:
vect_model = TfidfVectorizer(stop_words="english")

In [9]:
vect_mat = vect_model.fit_transform(df["short_description"])

In [10]:
cosine_sim = linear_kernel(vect_mat, vect_mat)

In [16]:
df.shape

(40000, 6)

In [20]:
indices = pd.Series(df.index,index=df["headline"]).drop_duplicates()

In [21]:
indices

headline
Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters                     0
American Airlines Flyer Charged, Banned For Life After Punching Flight Attendant On Video        1
23 Of The Funniest Tweets About Cats And Dogs This Week (Sept. 17-23)                            2
The Funniest Tweets From Parents This Week (Sept. 17-23)                                         3
Woman Who Called Cops On Black Bird-Watcher Loses Lawsuit Against Ex-Employer                    4
                                                                                             ...  
Trump Fails To Close Deal On Health Care Repeal                                              39995
Trump Escaped 'Wiretap' And Russia Questions In Local TV Interviews                          39996
‘Carmichael Show’ Creator Has Uncensored 'N-Word' In Upcoming Episode                        39997
Bryan Cranston Tells Pawternative Facts To Rehouse Adorable Shelter Puppies                  39998
O

In [26]:
def get_recomendations(headline, cosine_sim = cosine_sim):
    idx = indices[headline]
    sim_scores = enumerate(cosine_sim[idx])
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    
    sim_idx = [i[0] for i in sim_scores]
    print(df["headline"].iloc[sim_idx])
    

In [27]:
get_recomendations(df["headline"])

11910    Jimmy Kimmel Coaxes Wild 48-Hour Marriage Stor...
9099     Transphobic Congressional Hopeful Berates Pers...
1597     Justice Dept. To Prioritize Prosecuting Violen...
18911    Audio Of Corey Feldman Allegedly Naming Sexual...
35224       'New Girl' Squeezes Out One More Season On Fox
14032    Former Trump Aide Rick Gates Will Plead Guilty...
7374     Model And Actress Kim Porter Died From Pneumon...
39303    Video Shows Officer Casually Ignoring Gunshot ...
33146    Wreckage Of Missing Myanmar Military Plane Fou...
20068    Katy Perry Awarded $1.57 Million From Entrepre...
Name: headline, dtype: object
